In [120]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = load_dotenv("OPENAI_API_KEY")
from openai import OpenAI


Pretty Printing Helper  

In [121]:
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [122]:
client = OpenAI()
assistant = client.beta.assistants.create(
    name = "Math Tutor 2",
    instructions="You are a helpful personal math tutor, Answer questions and show your work.",
    model="gpt-4-1106-preview"
)
show_json(assistant)

{'id': 'asst_TFgnl18IgRYbNMpj6FF8ABh4',
 'created_at': 1699824018,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor 2',
 'object': 'assistant',
 'tools': []}

In [ ]:
thread = client.beta.threads.create()
show_json(thread)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I NEED to sovle this problem: 3x +11 = 14, can you help me ?",
)
show_json(message)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

Run is asynchronous operation. 

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

In [ ]:
message = client.beta.threads.messages.list(thread_id=thread.id)
show_json(message)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you explain this to me?",
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

wait_on_run(run, thread)
message = client.beta.threads.messages.list(thread_id=thread.id, order = "asc", after=message.id)
show_json(message)

FROM NOW ON, Create some function to make above code a lot cleaner  

In [123]:
MATH_ASSISTANT_ID = assistant.id

client = OpenAI()

def submit_message(assistant_id, thread,user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id,role="user",content=user_message)
    return client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant_id)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id,order="asc")

In [124]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

In [ ]:


#emulating concurrent users request

thread1, run1 = create_thread_and_run("I NEED to sovle this problem: 3x +11 = 14, can you help me ?")

thread2, run2 = create_thread_and_run("Could you explain linear algerbra to me ?")

thread3, run3 = create_thread_and_run("I don't like math, what can I DO ?")

In [ ]:
import time

def pretty_print(message):
    print("# Message")
    for m in message:
        print(f"{m.role}: {m.content[0].text.value}")

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    return run

run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

In [ ]:
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

## Adding Tool to the Assistant ###

In [ ]:
assistant = client.beta.assistants.update(MATH_ASSISTANT_ID,tools=[{"type":"code_interpreter"}])
show_json(assistant)

In [ ]:
thread, run = create_thread_and_run("Generate the first 20 fibbonaci numbers with code.")
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

#### Show the detailed steps

In [ ]:
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id, order="asc")

In [ ]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

## Retrival

In [ ]:
file = client.files.create(
    file=open(
        "data/language-models.pdf",
        "rb",
    ),
    purpose="assistants",
)

assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type":"code_interpreter"},{"type":"retrieval"}],
    file_ids=[file.id]
)
show_json(assistant)
    



In [ ]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences.")
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

### Functions

Let's take a look at an example by defining a display_quiz() Function for our Math Tutor.

This function will take a title and an array of questions, display the quiz, and get input from the user for each:

title
questions
    question_text
    question_type: [MULTIPLE_CHOICE, FREE_RESPONSE]
    choices: ["choice 1", "choice 2", ...]

In [125]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

Sample:


In [ ]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

In [126]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [127]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type":"code_interpreter"},
        {"type":"retrieval"},
        {"type":"function","function":function_json}
    ],
)
show_json(assistant)

{'id': 'asst_TFgnl18IgRYbNMpj6FF8ABh4',
 'created_at': 1699824018,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor 2',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
        'required': ['question_text']}}},
     'required': ['title', 'questions']},
    'desc

In [128]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)



In [129]:
run.status

'requires_action'

In [130]:
show_json(run)

{'id': 'run_JEWhfeOdFRUOvPYeQPI1PWU0',
 'assistant_id': 'asst_TFgnl18IgRYbNMpj6FF8ABh4',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699824089,
 'expires_at': 1699824689,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_NX7rRzFxYkA4CilVbn5IcXNa',
     'function': {'arguments': '{\n  "title": "Math Skills Assessment",\n  "questions": [\n    {\n      "question_text": "Explain why the sum of the angles in any triangle always equals 180 degrees.",\n      "question_type": "FREE_RESPONSE"\n    },\n    {\n      "question_text": "What is the value of \'x\' if 2x + 3 = 11?",\n      "question_type": "MULTIPLE_CHOICE",\n      "choices": ["4", "5", "3", "2"]\n    }\n  ]\n}',
      'name': 'display_quiz'},
     'type': 'function'}]},

## The required_action field indicates a Tool is waiting for us to run it and submit its output back to the Assistant

In [131]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)
print(f"Function name: {name}")
print(f"Arguments:")
arguments

Function name: display_quiz
Arguments:


{'title': 'Math Skills Assessment',
 'questions': [{'question_text': 'Explain why the sum of the angles in any triangle always equals 180 degrees.',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': "What is the value of 'x' if 2x + 3 = 11?",
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['4', '5', '3', '2']}]}

In [132]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Skills Assessment

Explain why the sum of the angles in any triangle always equals 180 degrees.

What is the value of 'x' if 2x + 3 = 11?
0. 4
1. 5
2. 3
3. 2

Responses: ["I don't know.", 'a']


In [133]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_JEWhfeOdFRUOvPYeQPI1PWU0',
 'assistant_id': 'asst_TFgnl18IgRYbNMpj6FF8ABh4',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699824089,
 'expires_at': 1699824689,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1699824089,
 'status': 'queued',
 'thread_id': 'thread_Y5ynIXxQ5Oh4TWe2pa7Jtrk3',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type'

In [134]:
run.status

'queued'

In [135]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Message
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: Thank you for participating in the quiz. Let's go through each question:

1. The question about why the sum of the angles in any triangle always equals 180 degrees is a fundamental concept in geometry. The sum of angles in a triangle is 180 degrees because of the way triangles fit into the Euclidean plane. One way to understand this is through the concept of parallel lines and transversals. When a transversal crosses two parallel lines, alternate interior angles are created on each side of the transversal. If you take one of the corners of a triangle and extend one of its sides, you create a straight line parallel to the opposite side of the triangle, and the angles thus created match up with the other two angles of the triangle. The straight line is 180 degrees, therefore the sum of the angles in the triangle must also be 180 degrees. Remember that th